In [1]:
# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#import nltk
import nltk
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('words')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize

#preprocessing
from nltk.corpus import stopwords  #stopwords
from nltk import word_tokenize,sent_tokenize # tokenizing
from nltk.stem import PorterStemmer,LancasterStemmer  # using the Porter Stemmer and Lancaster Stemmer and others
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer  # lammatizer from WordNet

# for named entity recognition (NER)
from nltk import ne_chunk

# vectorizers for creating the document-term-matrix (DTM)
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

#stop-words
stop_words_nl=set(nltk.corpus.stopwords.words('dutch'))
stop_words_en=set(nltk.corpus.stopwords.words('english'))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Irina\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Irina\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Irina\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Irina\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Irina\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [13]:
import os
import re

texts = []

for file in os.listdir("Summarization/summaries_txt"):
        if re.match(r"\d*\.txt", file):
            with open(f"Summarization/summaries_txt/{file}", 'r', encoding="latin-1") as text:
                texts.append(text.read())

print(len(texts))
print(texts[0])


768
De Wet tot wijziging van de Overeenkomst tussen het Koninkrijk België en de Staat Malta tot het vermijden van dubbele belasting en tot het voorkomen van het ontgaan van belasting, zoals gewijzigd door de Aanvullende Overeenkomst ondertekend te Brussel op 23 juni 1993, zal volkomengevolg hebben.


In [48]:
import spacy
import nl_core_news_lg

nlp = nl_core_news_lg.load()
# en_words = set(nltk.corpus.words.words())
# overly_abundant = ["article", "art", "publication"]
# months = ["january", "february", "march", "april", "may", "june", "july", "august", "september", "november", "december"]

def lemmatizer(texts):
    texts = [text.replace("\n", " ").strip() for text in texts]
    texts = [text.replace(";", " ").strip() for text in texts]
    texts = [text.replace(",", " ").strip() for text in texts]
    texts = [text.replace(".", " ").strip() for text in texts]
    docs = nlp.pipe(texts)
    cleaned_lemmas = [[t.lemma_ for t in doc if t.lemma_ not in stop_words_nl 
                        #and t.lemma_ in en_words
                        and t.lemma_ not in stop_words_en
                        #and t.lemma_.lower() not in overly_abundant
                        #and t.lemma_.lower() not in months
                        and not re.match(r".*\d.*", t.lemma_)] for doc in docs]

    return cleaned_lemmas

In [49]:
cleaned = []
parts = int(len(texts)/20)
for part in range(parts):
    print(f"Batch {part + 1} of {parts}")
    cleaned.append(lemmatizer(texts[20*part:20*(part+1)]))
#lemmatizer(texts)
print(len(cleaned))
print(cleaned[0][0])

Batch 1 of 38
Batch 2 of 38
Batch 3 of 38
Batch 4 of 38
Batch 5 of 38
Batch 6 of 38
Batch 7 of 38
Batch 8 of 38
Batch 9 of 38
Batch 10 of 38
Batch 11 of 38
Batch 12 of 38
Batch 13 of 38
Batch 14 of 38
Batch 15 of 38
Batch 16 of 38
Batch 17 of 38
Batch 18 of 38
Batch 19 of 38
Batch 20 of 38
Batch 21 of 38
Batch 22 of 38
Batch 23 of 38
Batch 24 of 38
Batch 25 of 38
Batch 26 of 38
Batch 27 of 38
Batch 28 of 38
Batch 29 of 38
Batch 30 of 38
Batch 31 of 38
Batch 32 of 38
Batch 33 of 38
Batch 34 of 38
Batch 35 of 38
Batch 36 of 38
Batch 37 of 38
Batch 38 of 38
38
['wet', 'wijziging', 'overeenkomst', 'tussen', 'Koninkrijk', 'België', 'staat', 'Malta', 'vermijden', 'dubbel', 'belasting', 'voorkomen', 'ontgaan', 'belasting', ' ', 'zoals', 'wijzigen', 'Aanv_ullend', 'overeenkomst', 'ondertekenen', 'Brussel', 'juni', ' ', 'zullen', 'volkomengevolg']


In [36]:

print(len(cleaned[0]))

20


In [50]:
vect = TfidfVectorizer(stop_words=stop_words_nl,max_features=1000)

In [51]:
cleaned_joined = []
    
[[cleaned_joined.append(" ".join(words)) for words in batch] for batch in cleaned]
print(len(cleaned_joined))

760


In [32]:
len(cleaned_joined)

760

In [52]:
vectorised = vect.fit_transform(cleaned_joined)
vect.get_feature_names()

['aan_bieden',
 'aan_brengen',
 'aan_kondigen',
 'aan_nemen',
 'aan_pakken',
 'aan_stellen',
 'aan_trekken',
 'aan_vangen',
 'aan_vullen',
 'aandeel',
 'aangelegenheid',
 'aangift_eformulier',
 'aangifte',
 'aangiftefo_rmulier',
 'aankoop',
 'aanleiding',
 'aanmerking',
 'aanpassing',
 'aanslagjaar',
 'aantal',
 'aanvraag',
 'aanvulling',
 'aanzienlijk',
 'aard',
 'abgeordnetenkammer',
 'accijnzen',
 'actie',
 'actief',
 'activiteit',
 'adjunct',
 'administratie',
 'administratief',
 'advies',
 'af',
 'af_leggen',
 'af_lopen',
 'af_nomen',
 'af_sluiten',
 'afdeling',
 'afgegeven',
 'afgekondigde',
 'afgevaardigden',
 'afgifte',
 'afhankelijk',
 'afkondiging',
 'afnemer',
 'aftrek',
 'afwezigheid',
 'afwijking',
 'afzondering',
 'airport',
 'akkoord',
 'akt',
 'akte',
 'algemeen',
 'algemeneadministratie',
 'alleen',
 'ambassadeur',
 'ambt',
 'ambtenaar',
 'ambtenarenzaken',
 'analyse',
 'ander',
 'annuleren',
 'april',
 'arbeid_plaats',
 'arbeidsongeschikt',
 'arbeidsongeschiktheid',
 

In [53]:
idf=vect.idf_
dd=dict(zip(vect.get_feature_names(), idf))
l=sorted(dd, key=(dd).get)
# print(l)
print(l[0:10])

['besluit', 'regering', 'maatregel', 'wijziging', 'kader', 'ministerie', 'artikel', 'coronavirus', 'zaken', 'minister']


## LDA

In [54]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=42,max_iter=1) 

lda_top=lda_model.fit_transform(vectorised)

In [55]:
# most important words for each topic
vocab = vect.get_feature_names()

for i, comp in enumerate(lda_model.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")

Topic 0: 
procent belgisch tegemoetkoming ambassadeur woord vertegenwoordiger op_heffen commissie school_jaar franse 

Topic 1: 
grond_wet belgiã kamer tussen volksvertegenwoordiger akkoord situatie context aan_nemen strijd 

Topic 2: 
zelfstandig sociaal gunst invoering activiteit maatregel zelfstandigen overbruggingsrecht inzake promotie 

Topic 3: 
vlaams besluit onderwijs minister regering nationaal aantal koninklijk treffen beperken 

Topic 4: 
besluit wijziging ministerieel maatregel regering coronavirus koninklijk artikel kader ministerie 

Topic 5: 
maatregel brussels aantal gezondheidscrisis stellen kader huurder zaken beperken in_voeren 

Topic 6: 
regeling gemeenschappelijkegemeenschapscommissie opschorting wet vleugel ingevolge mei arbeidsongeschiktheid procedure verkiezing 

Topic 7: 
minister belasting zaken financiã waarde wetboek toegevoegde ministerie volksgezondheid justitie 

Topic 8: 
brussels regering hoofdstedelijk gewest waals definitief besluit gemeenschap kader

## BERTopic

In [56]:
from bertopic import BERTopic

topic_model = BERTopic(language="dutch", calculate_probabilities=True)
topics, probs = topic_model.fit_transform(cleaned_joined)

Downloading: 100%|██████████| 190/190 [00:00<00:00, 38.0kB/s]
Downloading: 100%|██████████| 3.79k/3.79k [00:00<00:00, 947kB/s]
Downloading: 100%|██████████| 229/229 [00:00<00:00, 32.7kB/s]
Downloading: 100%|██████████| 471M/471M [01:25<00:00, 5.48MB/s] 
Downloading: 100%|██████████| 53.0/53.0 [00:00<00:00, 10.6kB/s]
Downloading: 100%|██████████| 5.07M/5.07M [00:00<00:00, 5.26MB/s]
Downloading: 100%|██████████| 239/239 [00:00<00:00, 39.8kB/s]
Downloading: 100%|██████████| 9.08M/9.08M [00:01<00:00, 5.27MB/s]
Downloading: 100%|██████████| 480/480 [00:00<00:00, 68.6kB/s]
Downloading: 100%|██████████| 14.8M/14.8M [00:02<00:00, 5.45MB/s]


In [57]:
freq = topic_model.get_topic_info(); freq.head(5)

,Topic,Count,Name
0,-1,156,-1_definitief_aufgrund_artikel_financiã
1,0,148,0_coronavirus_ministerieel_maatregel_verspreiding
2,1,56,1_brussels_hoofdstedelijk_brussel_gewestelijk
3,2,49,2_franse_gemeenschap_kader_gezondhe_idscrisis
4,3,35,3_grond_wet_volksvertegenwoordiger_wet_kamer


In [68]:
topic_model.get_topic(5) 

[('waals', 0.17340796890446997),
 ('regering', 0.08778724665772046),
 ('waalse', 0.06920941104668597),
 ('decreet', 0.0578820190640589),
 ('bijzonder', 0.05216838715052308),
 ('macht', 0.05158371046397992),
 ('gewest', 0.04208492026763185),
 ('toekenning', 0.039684177372790036),
 ('besluit', 0.03755048680736918),
 ('twee', 0.0371823878731185)]

In [59]:
topic_model.visualize_topics()

In [60]:
topic_model.visualize_distribution(probs[200], min_probability=0.0015)

In [61]:
topic_model.visualize_hierarchy(top_n_topics=50)

In [63]:
topic_model.visualize_barchart(top_n_topics=7)